In [2]:
import datetime as dt
import meteomatics.api as api
import pandas as pd
import numpy as np

# Meteomatics api call

The following code can be used to call the meteomatics api.

### WeatherCall function
The WeatherCall function calls the Metomatics api and returns a dataframe with the results of the call.

**Notes on the WeatherCall function:**
* The Metomatics api returns time stamps for averages right bounded (the preceding 24h are taken into account) )in the API because of this the dates are shifted by one day. That means that the following value: 
      validdate;t_mean_2m_24h:F 
      2021-05-06T00:00:00Z;72.4
      represents the 24h hour mean temperature of May 5.
* WeatherCall fixes the date shift for each row returned in a new column called 'date'. The original column with the shifted dates is droped from the dataframe that is returned






 

In [3]:
def WeatherCall(location,parameters,time):
    
    ###Credentials:
    username = 'smu_student'
    password = 'KExx3Nhs768c'
    
    #unpack time
    startdate,enddate,interval = time
    
    #empty results dataframe will be populated by query results
    results = pd.DataFrame()

    #for loop that loops through a list of latitude and longitude locations 
    for x in location:
        try:
            ts = api.query_time_series(x, startdate, enddate, interval,parameters, username, password) #query api
            ts_df = ts.reset_index() #lat, lon, and validdate come back as a multi Index this makes them columns
            results = results.append(ts_df) #append the results dataframe with query results
            results['date'] = results.validdate - dt.timedelta(days=1) #daily results are off by a day


        except Exception as e:
            print("Failed, the exception is {}".format(e)) 
        
    results = results.drop(columns=['validdate'])
    return results    

### Import mgrs location csv and format to work with WeatherCall function

The meteomatics api can only be queried for 10 locations at a time. To return results for all 385 mgrs locations the latitude and longitude have to be combined into individual records (tuples) and then sent through the api ten at a time. The following code will pull down the mgrs_final.csv file from our git convert the lat and long to tuples and put them in a list. The code will then split the orginal list into a list of lists each of the smaller lists containing ten locations except for the last list which only contains five locations. The code returns 'mgrs_locations' that can be passed into the 'WeatherCall' function to get results for all 385 locations at once. 

**<font color=red>Notes on location READ ME: </font>**
   * When 'mgrs_locations' list is sent through the 'WeatherCall' function 39 calls will be made to the api! One call = list of 10 locations

  * When 'mgrs_test' list is sent through the 'WeatherCall funciton 2 calls will be made to the api. Use this for testing until you're ready to get data for all lcoations.

In [4]:
from itertools import islice

#Import mgrs locations
mgrs = pd.read_csv('https://raw.githubusercontent.com/jbalson0808/fires/main/mgrs_final.csv') #set location of mgrs_final

#Select lat and longitude columns
mgrs = mgrs[['lat_10km','lon_10km']]

#Convert mgrs Lat and lon into a list of tuples 
records = mgrs.to_records(index=False)

#convert 
mgrs_list = list(records)

#split list into 39 smaller lists 
length_to_split = [10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,6]

inputt = iter(mgrs_list)

# All mgrs locations ################
mgrs_locations = [list(islice(inputt,elem)) for elem in length_to_split]


# mgrs_test this limits the locations to twenty and only makes 2 calls to the api useful in testing#######
mgrs_test = mgrs_locations[:2]

#print(mgrs_locations[:2]

In [30]:
mgrs_locations[:2]

[[(33.70526732, -118.079125),
  (33.79544412, -118.0802553),
  (33.88561957, -118.0813907),
  (33.97579365, -118.0825311),
  (34.06596638, -118.0836766),
  (34.15613776, -118.0848272),
  (34.24630777, -118.085983),
  (33.61598164, -117.970214),
  (33.70616282, -117.9712268),
  (33.79634266, -117.9722441)],
 [(33.88652115, -117.973266),
  (33.97669829, -117.9742925),
  (34.06687408, -117.9753235),
  (34.15704851, -117.9763591),
  (34.24722159, -117.9773993),
  (33.6167802, -117.8624237),
  (33.70696409, -117.8633241),
  (33.79714665, -117.8642284),
  (33.88732786, -117.8651368),
  (33.97750773, -117.8660493)]]

### Set Start date and enddate of call
Use the code below to set the start and end dates for the weather data you want to call. The start date, end date, and interval are packed into a variable called 'time' that can be passed to the WeatherCall function. 

Notes on start and end date: 
* the Meteomatics api uses right bounded dates for averages when entering start date and end date you need to shift the dates by one day. Example, if you want to get the daily mean (00:00 until 00:00 UTC) temperatures between May 5 and 10, you should specify the following:
        startdate_ts = dt.datetime(2021,5,6)
        enddate_ts = dt.datetime(2021,5,11)
        interval_ts = dt.timedelta(hours=24) 
        
* The code below shifts the start and end dates you enter by one day to make it easier to use. Just enter the actual dates you want and the function will do the rest


In [13]:
# month_s = 1    #start date month
# day_s = 1      #start date day
# year_s = 2000  #start date year

# month_e = 12    #end date month
# day_e = 31     #end date day
# year_e = 2010  #end date year

month_s = 1    #start date month
day_s = 1      #start date day
year_s = 2011  #start date year

month_e = 12    #end date month
day_e = 31     #end date day
year_e = 2015  #end date year

########### function to set WeatherCall time ##########################
def t(year_s,month_s,day_s,year_e,month_e,day_e):
    
    #start date +1day
    startdate = dt.datetime(year_s,month_s,day_s) + dt.timedelta(days=1) 
    
    #end date +1day
    enddate = dt.datetime(year_e,month_e,day_e) + dt.timedelta(days=1) 
    
    #interval hours = 24 for daily data
    interval = dt.timedelta(hours=24)
    
    time = startdate,enddate,interval
    
    return time
########################################################################

time = t(year_s,month_s,day_s,year_e,month_e,day_e)

### Set parameters you want to return

Use the code below to set the values of the parameters you want the api to return.Temperature and other parameters available that can be queried for 24 h periods, like dew point and relative humidity also 24 h precipitation sums can be queried. 
* Parameters that can be called are found in the Metomatics Documentation here: https://www.meteomatics.com/en/api/available-parameters/

In [5]:
parameters = ['t_mean_2m_24h:F', 't_min_2m_24h:F','t_max_2m_24h:F','precip_24h:mm','sunshine_duration_24h:min','drought_index:idx','soil_moisture_index_-15cm:idx','soil_type:idx','wind_speed_2m:mph','elevation:m']

# Call Meteomatics API with WeatherCall

**Run the code below to call the Metomatics API after setting location, time, and parameters above.**


In [6]:
#Call API
# results = WeatherCall(mgrs_test,parameters,time)
results = WeatherCall(mgrs_locations,parameters,time)

#View Results
results.head()

,lat,lon,t_mean_2m_24h:F,t_min_2m_24h:F,t_max_2m_24h:F,precip_24h:mm,sunshine_duration_24h:min,drought_index:idx,soil_moisture_index_-15cm:idx,soil_type:idx,wind_speed_2m:mph,elevation:m,date
0,33.705267,-118.079125,53.0,47.6,58.5,0.45,232.7,-4.0,0.0,2.0,5.6,0.0,2000-01-01 00:00:00+00:00
1,33.705267,-118.079125,54.2,49.6,60.6,0.08,572.1,-4.0,0.0,2.0,9.6,0.0,2000-01-02 00:00:00+00:00
2,33.705267,-118.079125,53.2,42.7,66.0,0.00,598.0,-4.0,0.0,2.0,4.5,0.0,2000-01-03 00:00:00+00:00
3,33.705267,-118.079125,55.1,44.2,67.5,0.00,598.9,-4.0,0.0,2.0,6.2,0.0,2000-01-04 00:00:00+00:00
4,33.705267,-118.079125,54.9,44.0,68.9,0.00,598.5,-4.0,0.0,2.0,3.0,0.0,2000-01-05 00:00:00+00:00


In [40]:
results

,lat,lon,t_mean_2m_24h:F,t_min_2m_24h:F,t_max_2m_24h:F,precip_24h:mm,sunshine_duration_24h:min,drought_index:idx,soil_moisture_index_-15cm:idx,soil_type:idx,wind_speed_2m:mph,elevation:m,date
0,33.705267,-118.079125,53.0,47.6,58.5,0.45,232.7,-4.0,0.000,2.0,5.6,0.0,2000-01-01 00:00:00+00:00
1,33.705267,-118.079125,54.2,49.6,60.6,0.08,572.1,-4.0,0.000,2.0,9.6,0.0,2000-01-02 00:00:00+00:00
2,33.705267,-118.079125,53.2,42.7,66.0,0.00,598.0,-4.0,0.000,2.0,4.5,0.0,2000-01-03 00:00:00+00:00
3,33.705267,-118.079125,55.1,44.2,67.5,0.00,598.9,-4.0,0.000,2.0,6.2,0.0,2000-01-04 00:00:00+00:00
4,33.705267,-118.079125,54.9,44.0,68.9,0.00,598.5,-4.0,0.000,2.0,3.0,0.0,2000-01-05 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20085,33.167642,-117.536254,55.9,51.7,60.6,0.00,470.9,4.0,0.889,2.0,4.6,0.0,2010-12-27 00:00:00+00:00
20086,33.167642,-117.536254,56.4,50.3,60.7,0.00,466.8,4.0,0.902,2.0,3.9,0.0,2010-12-28 00:00:00+00:00
20087,33.167642,-117.536254,56.5,54.6,60.1,16.65,184.8,4.0,1.027,2.0,15.1,0.0,2010-12-29 00:00:00+00:00
20088,33.167642,-117.536254,52.9,49.8,57.7,0.00,584.7,4.0,0.942,2.0,6.4,0.0,2010-12-30 00:00:00+00:00


In [7]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1546930 entries, 0 to 20089
Data columns (total 13 columns):
 #   Column                         Non-Null Count    Dtype              
---  ------                         --------------    -----              
 0   lat                            1546930 non-null  float64            
 1   lon                            1546930 non-null  float64            
 2   t_mean_2m_24h:F                1546930 non-null  float64            
 3   t_min_2m_24h:F                 1546930 non-null  float64            
 4   t_max_2m_24h:F                 1546930 non-null  float64            
 5   precip_24h:mm                  1546930 non-null  float64            
 6   sunshine_duration_24h:min      1546930 non-null  float64            
 7   drought_index:idx              1546930 non-null  float64            
 8   soil_moisture_index_-15cm:idx  1462552 non-null  float64            
 9   soil_type:idx                  1462552 non-null  float64            
 

In [8]:
len(results.date.unique())

4018

In [14]:
time

(datetime.datetime(2011, 1, 2, 0, 0),
 datetime.datetime(2016, 1, 1, 0, 0),
 datetime.timedelta(days=1))

In [15]:
#Call API
# results = WeatherCall(mgrs_test,parameters,time)
results2 = WeatherCall(mgrs_locations,parameters,time)

#View Results
results2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703010 entries, 0 to 9129
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype              
---  ------                         --------------   -----              
 0   lat                            703010 non-null  float64            
 1   lon                            703010 non-null  float64            
 2   t_mean_2m_24h:F                703010 non-null  float64            
 3   t_min_2m_24h:F                 703010 non-null  float64            
 4   t_max_2m_24h:F                 703010 non-null  float64            
 5   precip_24h:mm                  703010 non-null  float64            
 6   sunshine_duration_24h:min      703010 non-null  float64            
 7   drought_index:idx              703010 non-null  float64            
 8   soil_moisture_index_-15cm:idx  649271 non-null  float64            
 9   soil_type:idx                  650737 non-null  float64            
 10  wind_speed

In [16]:
results2.head()

,lat,lon,t_mean_2m_24h:F,t_min_2m_24h:F,t_max_2m_24h:F,precip_24h:mm,sunshine_duration_24h:min,drought_index:idx,soil_moisture_index_-15cm:idx,soil_type:idx,wind_speed_2m:mph,elevation:m,date
0,33.705267,-118.079125,49.9,42.9,57.0,0.00,576.4,4.0,0.844,2.0,1.9,0.0,2011-01-01 00:00:00+00:00
1,33.705267,-118.079125,50.8,46.2,55.7,3.80,204.0,4.0,0.817,2.0,6.6,0.0,2011-01-02 00:00:00+00:00
2,33.705267,-118.079125,50.8,46.7,57.0,11.71,152.0,4.0,0.940,2.0,2.1,0.0,2011-01-03 00:00:00+00:00
3,33.705267,-118.079125,52.9,44.6,62.9,0.65,577.1,4.0,0.911,2.0,3.4,0.0,2011-01-04 00:00:00+00:00
4,33.705267,-118.079125,53.3,43.8,64.0,0.00,588.0,4.0,0.871,2.0,4.0,0.0,2011-01-05 00:00:00+00:00


In [52]:
weather = pd.concat([results, results2], ignore_index=True)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2249940 entries, 0 to 2249939
Data columns (total 13 columns):
 #   Column                         Dtype              
---  ------                         -----              
 0   lat                            float64            
 1   lon                            float64            
 2   t_mean_2m_24h:F                float64            
 3   t_min_2m_24h:F                 float64            
 4   t_max_2m_24h:F                 float64            
 5   precip_24h:mm                  float64            
 6   sunshine_duration_24h:min      float64            
 7   drought_index:idx              float64            
 8   soil_moisture_index_-15cm:idx  float64            
 9   soil_type:idx                  float64            
 10  wind_speed_2m:mph              float64            
 11  elevation:m                    float64            
 12  date                           datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(12)
mem

In [60]:
weather.to_csv('2000to2015_Weather_v2.csv')

In [44]:
# import mgrs
# m = mgrs.MGRS() #mgrs API
# to_mgrs_10km = []  #precision 1
# to_mgrs_100km = []  #precision 0

# for i, j in zip(weather['lat'].tolist(), np.round(weather['lon'],8).tolist()):
#     to_mgrs_10km.append(m.toMGRS(i, j, MGRSPrecision=1)) 
#     to_mgrs_100km.append(m.toMGRS(i, j, MGRSPrecision=0)) # use mgrs API to convert lat/lon to MGRS conventions

# weather['mgrs_10km'] = to_mgrs_10km
# weather['mgrs_100km'] = to_mgrs_100km

In [53]:
# weather.info()

In [46]:
# len(weather.mgrs_10km.unique())

274

In [59]:
# len(weather.mgrs_100km.unique())

In [54]:
# weather.mgrs_100km.unique()

In [55]:
# weather.head()

In [56]:

# # w_lat, w_lon = [], []
# w_coords = []

# for i,j in zip(weather.lat.unique().tolist(), np.round(weather.lon,8).unique().tolist()):
#     # print(i,j)
#     w_coords.append([i,j])

# w_coords[:20]

In [57]:
# len(w_coords)

In [58]:
# mgrs_locations[:2]